In [340]:
import geohash
import pandas as pd
import configparser
import googlemaps
import time
import json
import os
import networkx as nx
from networkx.readwrite import json_graph
from bs4 import BeautifulSoup
import requests
import csv
from math import radians, cos, sin, asin, sqrt

In [6]:
def odd_even(str_hash):
    if len(str_hash)%2 == 0:
        return 'even'
    else:
        return 'odd'

In [288]:
config = configparser.ConfigParser()
config.read("config.ini")
API_key = config['Keys']['google_API']
gmaps = googlemaps.Client(key=API_key)()

In [289]:
def get_geohash_distance(gh_A,gh_B):   
    GPS_A = geohash.decode(gh_A)
    GPS_B = geohash.decode(gh_B) 
    directions_result = gmaps.directions(GPS_A,
                                         GPS_B,
                                         mode="driving")
    time.sleep(1)
    return directions_result[0]['legs'][0]['distance']['value'] 

In [400]:
GEOHASH_PRECISION = 3
MAX_RANGE = 426 #Maximum Tesla Model S range in Km from https://en.wikipedia.org/wiki/Tesla_Model_S
POP_DICT = load_pop_dict()
def build_connections(G,src_hash):
    print (nx.number_of_nodes(G))
    connections = {}
    node_hashes = ([node for node in G
                  if node[0:GEOHASH_PRECISION] in geohash.expand(src_hash[0:GEOHASH_PRECISION])
                  and node != src_hash])
    close_connections = ([{'node':node_gh,
                           'distance':get_geohash_distance(src_hash,node_gh)} for node_gh in node_hashes])
    for connection in close_connections:
        if connection['distance']/1000 <= MAX_RANGE:
            edge_weight = get_edge_weight(G,src_hash,connection['node'])
            G.add_edge(src_hash,connection['node'],{'weight':edge_weight,'distance':connection['distance'],
                                                    'lon_lat_1':reverse_GPS(geohash.decode(src_hash)),
                                                    'lon_lat_2':reverse_GPS(geohash.decode(connection['node']))})
    return G

In [398]:
def get_edge_weight(G,src_hash,connection_hash):
    try:
        pop1 = get_closest_population(src_hash,8)
        pop2 = get_closest_population(connection_hash,8)
        return (pop1+pop2)/POP_DICT['total']
    except KeyError as e:
        print(e)
        return 0    

In [9]:
def reverse_GPS(GPS):
    return [GPS[1],GPS[0]]

In [404]:
def build_network():
    G = load_network()
    df = pd.read_csv("Teslarati_SC_data.csv",dtype={'Stalls': float,'Zip':str,'Tesla':str,'Elev':str})
    df["lat"], df["lon"] = zip(*df["GPS"].str.split(',').tolist())
    df["lat"], df["lon"] = df["lat"].astype(float), df["lon"].astype(float)
    df['GPS_lon_lat'] = df.apply(lambda x: [x["lon"],x["lat"]], axis=1)
    df['geohash'] = df.apply(lambda x: geohash.encode(x['lat'],x['lon']), axis=1)
    df['SC_data'] = df.apply(lambda x: parse_Tesla_SC_data(x['Tesla']), axis=1)
    
    for i in df['geohash'].keys():
        if df['geohash'][i] not in G:
            G.add_node(df['geohash'][i],{key:df[key][i] for key in df.keys()})
            build_connections(G,df['geohash'][i])
    
    network = json_graph.node_link_data(G)
    with open("network.json","w") as f:
        json.dump(network,f)
    return G

In [405]:
test_g = build_network()

11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
2

In [282]:
def load_network():
    if os.path.getsize("network.json") > 0:
        with open("network.json","r") as f:
            data = json.load(f)
            G = json_graph.node_link_graph(data)
    else:
        G=nx.Graph()
    return G

In [353]:
def load_pop_dict():
    with open('populations.json', 'r') as f:
        p_dict = json.load(f)
    return p_dict

In [183]:
def parse_Tesla_SC_data(URL):
    SC_data = {}
    r = requests.get(URL)
    soup = BeautifulSoup(r.text,"html.parser")
    attr_lists = soup.find_all('p')
    for attr in attr_lists:
        if attr.find('strong'):
            #probably not ideal, but only way I could get BS to parse 'br' correctly. Better way must exist.
            SC_data[attr.find('strong').text] = [value for value in attr.childGenerator()
                                                 if value.name == None and value != ' ']
    time.sleep(1)
    return SC_data

In [145]:
parse_Tesla_SC_data('http://www.teslamotors.com/findus/location/supercharger/barstowsupercharger')

{'Amenities': ['Tanger Outlet mall, take the foot path around the AM/PM'],
 'Charging': ['8 Supercharger stalls, available 24/7'],
 'Public Charging': ['Pay NEMA 14-50 Outlet',
  'Barstow Calico KOA',
  '35250 Outer Highway 15 North',
  'Yermo, CA 92398'],
 'Restrooms': ["Chili's", 'Country Inn (open 24hrs)'],
 'Wifi': ['Country Inn',
  'Starbucks (Starbucks is located behind Country Inn, not visible from the charging stalls)']}

In [323]:
def google_city_location(city):
    location = gmaps.geocode(city)
    time.sleep(.1)
    return location

In [313]:
google_city_GPS('WasagaBeach_ON')[0]['geometry']['location']

{'lat': 44.5207419, 'lng': -80.01606679999999}

In [396]:
def build_pop_dict():
    total_pop = 0
    pop_gps_dict = {}
    for city,pop in POP_DICT.items():
        try:
            geocode_data = google_city_GPS(city)
            city_location = geocode_data[0]['geometry']['location']
            gh = geohash.encode(city_location['lat'],city_location['lng'])
            pop_gps_dict[gh] = ({'city':city,
                                        'population':pop,
                                        'lat':city_location['lat'],
                                        'lon':city_location['lng']})
            total_pop = total_pop + pop
        except IndexError as e:
            print (geocode_data)
            print (city + " not found")
    pop_gps_dict['total'] = total_pop
    with open("populations.json","w") as f:
        json.dump(pop_gps_dict,f)    

In [393]:
def get_closest_population(src_hash,precision):
    close_gh = ([gh for gh in list(POP_DICT.keys())
                    if gh[0:precision] in geohash.expand(src_hash[0:precision])])
    if close_gh:
        if len(close_gh) == 1:
            return POP_DICT[close_gh[0]]['population']
        else:
            src_GPS = reverse_GPS(geohash.decode(src_hash))
            gh_distances = [(gh,haversine(*src_GPS,*reverse_GPS(geohash.decode(gh)))) for gh in close_gh]
            closest_gh = sorted(gh_distances,key=lambda tup: tup[1])[0][0]
            return POP_DICT[closest_gh]['population']
    else:
        return get_closest_population(src_hash,precision-1)

In [361]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

In [395]:
get_closest_population('dnq2u3y4b',8)

10081